In [12]:
import torch
import torch.nn.functional as F
import torchvision.models as models
from jupyterplot import ProgressPlot
from tqdm.notebook import tqdm

from Define import *
import NUS_WIDE as nus
import TenNet as TenNet
import PredictionModel as P
from TripletLossFunc import TripletLossFunc

In [13]:
getTenNetFromFile = False
getDecoderFromFile = False
test = False

In [ ]:
if test or device == torch.device('cpu'):
    Number_Of_Images_Train = 100
    Number_Of_Images_Valid = 100
    N_Epochs = 1
    N_Epochs_Decoder = 1
    
#NUS_WIDE
train_data = nus.NUS_WIDE_Helper(nus.DataSetType.Train_81, Number_Of_Images_Train)
valid_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_81,  Number_Of_Images_Valid)
#test_data = NUS_WIDE_Helper(DataSetType.Test_81)

batch_size = BATCH_SIZE
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=batch_size)
#test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
n_epochs = N_Epochs
Lambda = 0.01

if test or device == torch.device('cpu'):
    image_model = models.vgg11(pretrained=True).to(device)
else:
    image_model = models.vgg16(pretrained=True).to(device) 
#image_model = TenNet_Image().to(device)
tag_model = TenNet.TenNet_Tag(train_data.get_tag_list()).to(device)

#triplet_loss = TripletLossFunc(Margin_Distance)
triplet_loss = TripletLossFunc(Margin_Distance, hybrid=True)

name="SavedModelState/IT_model.ckpt"
ten_res = TenNet.run(image_model,tag_model,train_loader,valid_loader,triplet_loss,Lambda,n_epochs,test,getTenNetFromFile,name)

In [ ]:
res = ten_res
#TenNet.printLossLog(res, n_epochs)
TenNet.printLossProgressPlot(res, n_epochs)
TenNet.printDistanceProgressPlot(res, n_epochs, train=True)
TenNet.printDistanceProgressPlot(res, n_epochs, train=False)

In [ ]:
#loss_funk = torch.nn.MSELoss(reduce=False, size_average=False)
loss_funk = F.pairwise_distance

n_epochs = 60
threshold = 0.5

In [ ]:
hybrid_model = P.TagDecoder(train_data.get_tag_num()).to(device)
mod = P.LossModel.HybridModel
hybrid_res = P.run(hybrid_model,mod,tag_model,image_model,train_loader,valid_loader,loss_funk,threshold,n_epochs,test,getDecoderFromFile)

In [ ]:
res = hybrid_res
#P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)

In [ ]:
predict_model = P.TagDecoder(train_data.get_tag_num()).to(device)
mod = P.LossModel.PredictModel
predict_res =P.run(predict_model,mod,tag_model,image_model,train_loader,valid_loader,loss_funk,threshold,n_epochs,test,getDecoderFromFile)

In [ ]:
res = predict_res
#P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)

In [ ]:
decoder_model = P.TagDecoder(train_data.get_tag_num()).to(device)
mod = P.LossModel.DecoderModel
decoder_res =P.run(decoder_model,mod,tag_model,image_model,train_loader,valid_loader,loss_funk,threshold,n_epochs,test,getDecoderFromFile)

In [ ]:
res = decoder_res
#P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)